In [1]:
import pandas as pd
import geohash2 as gh
from tqdm import tqdm
import json
from datetime import datetime

In [2]:
data = pd.read_csv('../model/gps_data2.csv')

def create_geohash(lat, lon):
	return gh.encode(lat, lon, precision=7)

In [3]:
data['Geohash'] = data.apply(lambda x: create_geohash(x['Latitude'], x['Longitude']), axis=1)

In [4]:
grouped_data = data.groupby('Geohash').agg({
	'Taxi ID': lambda x: list(set(x)),
	'Timestamp': list,
	'Latitude': 'mean',
	'Longitude': 'mean'
}).reset_index()

In [5]:
grouped_data

,Geohash,Taxi ID,Timestamp,Latitude,Longitude
0,wx4d5u6,[7653],[2008-02-05 07:39:15],39.750450,116.184500
1,wx4d5vm,[4657],[2008-02-07 13:14:21],39.756230,116.187550
2,wx4d5wz,"[9921, 5442, 9109]","[2008-02-06 19:53:52, 2008-02-03 08:19:30, 200...",39.764067,116.180303
3,wx4d5xz,"[10282, 4476, 861]","[2008-02-05 13:45:33, 2008-02-06 00:04:20, 200...",39.769920,116.180210
4,wx4d5y8,"[224, 10177, 9030, 8170, 9259, 2162, 6548, 373...","[2008-02-02 16:44:57, 2008-02-04 07:10:42, 200...",39.763283,116.181595
...,...,...,...,...,...
53872,wx4uj58,[8634],[2008-02-08 08:53:01],40.097570,116.587370
53873,wx4uj5p,[8833],"[2008-02-04 03:03:09, 2008-02-04 03:03:09]",40.094930,116.597580
53874,wx4uj5x,[133],"[2008-02-08 17:04:45, 2008-02-08 17:04:45]",40.097640,116.597500
53875,wx4uj61,"[3772, 2446]","[2008-02-02 14:10:44, 2008-02-03 14:21:07]",40.089810,116.599500


In [6]:
location_pairs = []

for _, row in tqdm(grouped_data.iterrows()):
	geohash = row['Geohash']
	person_ids = row['Taxi ID']
	timestamps = row['Timestamp']
	latitude = row['Latitude']
	longitude = row['Longitude']
	
	for i in range(len(person_ids)):
		for j in range(i + 1, len(person_ids)):
			pair = {
				'Geohash': geohash,
				'Person ID 1': person_ids[i],
				'Person ID 2': person_ids[j],
				'Timestamps': {
					person_ids[i]: timestamps[i],
					person_ids[j]: timestamps[j]
				},
				'Latitude': latitude,
				'Longitude': longitude
			}
			location_pairs.append(pair)

0it [00:00, ?it/s]

7774it [05:32, 11.67it/s] 

In [ ]:
with open('../files/edges/new_tdrive.json', 'w') as json_file:
	json.dump(location_pairs, json_file, indent=4)